## 추론(예측) 기반 기법과 신경망

## word2vec : 워드투벡터
2013년 구글의 토마스미콜로프(Tomas Mikolov)의 팀이 개발<br>
<b>word2vec</b> 알고리즘은 <b>신경망 모델</b>을 사용 하여 큰 텍스트 코퍼스에서 단어 연관성을 학습. 학습이 끝나면 이러한 모델은 동의어 단어를 감지하거나 부분 문장에 대한 추가 단어를 제안 할 수 있다. word2vec는 <b>벡터</b> 라고하는 특정 숫자 목록을 사용하여 각각의 고유 한 단어를 나타낸다 . 벡터는 간단한 수학적 함수 ( 벡터 간의 코사인 유사성 ) 가 해당 벡터가 나타내는 단어 간의 의미 유사성 수준을 나타내 도록 신중하게 선택 된다.

### [1] 신경망에서의 단어 처리

In [9]:
import numpy as np

text = 'You say goodbye and I say hello.' 
# {0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}

# 여기서 'you'만  one-hot 인코딩으로 표현
c = np.array([[1,0,0,0,0,0,0]])   # (1,7)
print('c:\n',c)

W = np.random.randn(7,3)          # (7,3)
print('W:\n',W)

h = np.matmul(c,W)    # (1,7) * (7,3) = (1,3)
print('h:\n',h)

c:
 [[1 0 0 0 0 0 0]]
W:
 [[ 0.27057969  0.38373031 -1.70897451]
 [ 0.30866767  1.42067698 -0.34023722]
 [-1.20686166 -0.21024762  0.39679496]
 [ 0.99929066  0.60731328  0.6160202 ]
 [ 1.49960434  0.26850341 -1.03854504]
 [ 1.28078359 -2.38656342  0.07249293]
 [ 0.66537126 -0.57329423  0.43260154]]
h:
 [[ 0.27057969  0.38373031 -1.70897451]]


In [10]:
class MatMul:
    def __init__(self,W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.x = None
        
    def forward(self,x):
        W = self.params
        out = np.dot(x,W)
        self.x = x
        return out
        
    def backward(self,dout):
        W = self.params
        dx = np.dot(dout,W.T)
        dw = np.dot(self.x.T,dout)
        self.grads[0][...] = dw  # 깊은복사
        return dx

In [12]:
text = 'You say goodbye and I say hello.' 
# {0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}

# 여기서 'you'만  one-hot 인코딩으로 표현
c = np.array([[1,0,0,0,0,0,0]])   # (1,7)
print('c:\n',c)

W = np.random.randn(7,3)          # (7,3)
print('W:\n',W)

layer = MatMul(W)
h = layer.forward(c)
print('h:\n',h)

c:
 [[1 0 0 0 0 0 0]]
W:
 [[ 0.87247063  1.48548833  0.41054729]
 [-0.46347123  0.91113802  0.15378532]
 [-0.03223421 -0.56688987 -0.89673633]
 [-1.38118188 -0.28851011 -1.14560844]
 [ 1.04891105  0.41506112 -0.32765563]
 [-0.95723635  1.63278766  0.07933085]
 [ 0.64927423 -0.93273748  1.96082541]]
h:
 [[[0.87247063 1.48548833 0.41054729]]]


## [2] 단순한 word2vec

### CBOW (Continuous Bag of Words) 모델

#### Word2Vec에는 CBOW(Continuous Bag of Words)와 Skip-Gram 두 가지 방식이 있다
- $ CBOW $ 는 주변에 있는 단어들을 가지고, 중간에 있는 단어들을 예측하는 방법 <br>
  타깃(target)은 중앙 단어 그 주변 단어들이 맥락(contexts)이다
- $ Skip-Gram $ 은 중간에 있는 단어로 주변 단어들을 예측하는 방법

#### BOW(Bag of Words) : 단어들의 순서는 전혀 고려하지 않고, 단어들의 출현 빈도(frequency)에만 집중하는 텍스트 데이터의 수치화 표현 방법

BOW를 만드는 과정<br>
(1) 우선, 각 단어의 고유한 인덱스(Index)를 부여한다.<br>
(2) 각 인덱스의 위치에 단어 토큰의 등장 횟수를 기록한 벡터(Vector)를 만든다.<br>

"정부가 발표하는 물가상승률과 소비자가 느끼는 물가상승률은 다르다."<br>
('정부': 0, '가': 1, '발표': 2, '하는': 3, '물가상승률': 4, '과': 5, '소비자': 6, '느끼는': 7, '은': 8, '다르다': 9) <br>
BOW: [1, 2, 1, 1, 2, 1, 1, 1, 1, 1]  ==> '가' 와 '물가상승률' 은 2회 발생